In [1]:
import polars as pl
from src.paths import PYTHON_VULNERABILITY_FIXES_DATA_PATH
from github import Github, Auth
from os import environ
import gitlab
import re
import jedi
import whatthepatch
from git import Repo
from collections import defaultdict
from pathlib import Path
from typing import Any
import jedi.api
import jedi.common
import os
import traceback


## Filter out data

In [2]:
# python_commit_data = commit_data_only_top_langs.filter(pl.col("language") == "Python")
python_vulnerability_fixes = pl.read_parquet(PYTHON_VULNERABILITY_FIXES_DATA_PATH)
print(python_vulnerability_fixes.unique("vulnerability_id").shape[0])
python_vulnerability_fixes = python_vulnerability_fixes.group_by('vulnerability_id', 'repo', 'commit', 'pull_request_number', 'file', 'patch', 'patch_time', 'commit_source', 'file_extension', 'language').agg(pl.col("cwe_id"))
python_vulnerability_fixes = python_vulnerability_fixes.unique("patch")
print(python_vulnerability_fixes.unique("vulnerability_id").shape[0])


1805
1802


## Filter out data connected with test functionallity

In [3]:
python_vulnerability_fixes = python_vulnerability_fixes.filter(
    (
        pl.col("file").str.contains(r"\/{0,1}[tT][eE][sS][tT][sS]{0,1}\/")
        | pl.col("patch").str.contains("pytest")
        | pl.col("patch").str.contains("unittest")
    ).not_()
)
python_vulnerability_fixes.unique("vulnerability_id").shape[0]

1795

## Process missing commit

In [4]:
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))
python_vulnerability_fixes = python_vulnerability_fixes.with_columns(
    pl.when(pl.col("pull_request_number") == 24391)
    .then(pl.lit("86664c9405136a4904775c52e6caf100a474ec58"))
    .otherwise(pl.col("commit"))
    .alias("commit")
)
print(python_vulnerability_fixes.filter(pl.col("commit").is_null()))


shape: (1, 11)
┌────────────┬────────────┬────────┬────────────┬───┬───────────┬───────────┬──────────┬───────────┐
│ vulnerabil ┆ repo       ┆ commit ┆ pull_reque ┆ … ┆ commit_so ┆ file_exte ┆ language ┆ cwe_id    │
│ ity_id     ┆ ---        ┆ ---    ┆ st_number  ┆   ┆ urce      ┆ nsion     ┆ ---      ┆ ---       │
│ ---        ┆ str        ┆ str    ┆ ---        ┆   ┆ ---       ┆ ---       ┆ str      ┆ list[str] │
│ str        ┆            ┆        ┆ i64        ┆   ┆ str       ┆ str       ┆          ┆           │
╞════════════╪════════════╪════════╪════════════╪═══╪═══════════╪═══════════╪══════════╪═══════════╡
│ 2021-3733  ┆ python/cpy ┆ null   ┆ 24391      ┆ … ┆ github    ┆ py        ┆ Python   ┆ ["CWE-400 │
│            ┆ thon       ┆        ┆            ┆   ┆           ┆           ┆          ┆ "]        │
└────────────┴────────────┴────────┴────────────┴───┴───────────┴───────────┴──────────┴───────────┘
shape: (0, 11)
┌─────────────┬──────┬────────┬─────────────┬───┬────────────

In [5]:
auth = Auth.Token(environ["GITHUB_TOKEN"])
github_client = Github(auth=auth)
gl = gitlab.Gitlab(private_token=environ["GITLAB_TOKEN"])


In [ ]:
def clear_file_content(content: str):
    return re.sub(r"#.+\n", "\n", content)


def clear_file_content_v2(content: str):
    content = re.sub(r"#.+\n", "\n", content)
    return re.sub(r"\n\n", "\n", content)


def read_lines(file: str, line_numbers: set[int]):
    with open(file) as f:
        return [
            line for i, line in enumerate(f.readlines(), start=1) if i in line_numbers
        ]


def get_function_body_lines(function: jedi.api.classes.Name) -> set[int]:
    body_lines: set[int] = set()
    start = function.get_definition_start_position()
    end = function.get_definition_end_position()
    if start is None or end is None:
        if function.type == "module": 
            return set()
        print(function.get_line_code())
        print(function.module_path)
        print(function)
        raise ValueError("Function definition not found")
    line_numbers = list(range(start[0], end[0] + 1))
    body_lines.update(line_numbers)

    parent = function.parent()
    while parent:
        if parent.type == "class" or parent.type == "function":
            start = parent.get_definition_start_position()
            end = parent.defined_names()[0].get_definition_start_position()
            if start is None or end is None:
                print(function.get_line_code())
                print(function)
                raise ValueError("Class definition not found")
            line_numbers = list(range(start[0], end[0]))
            body_lines.update(line_numbers)
        elif parent.type == "module":
            break
        else:
            print(parent.module_path)
            print(parent.get_definition_start_position())
            raise NotImplementedError(f"Parent type {parent.type}")
        parent = parent.parent()

    return body_lines


def get_function_context(
    script: jedi.Script, function: jedi.api.classes.Name
) -> dict[Path, set[int]]:
    if function.module_path is None:
        raise ValueError("Module path not found")
    function_body_lines: set[int] = set()
    start = function.get_definition_start_position()
    end = function.get_definition_end_position()
    if start is None or end is None:
        raise ValueError("Function definition not found")

    function_code = function.get_line_code(after=end[0] - start[0])
    context: dict[Path, set[int]] = defaultdict(set)
    for line_number, line_code in enumerate(function_code.split("\n"), start=start[0]):
        function_body_lines.add(line_number)
        for column_number, _ in enumerate(line_code, start=1):
            names: list[jedi.api.classes.Name] = script.goto(
                line_number, column_number, follow_imports=True
            )

            names_no_follow: list[jedi.api.classes.Name] = script.goto(
                line_number,
                column_number,
            )
            for name in names_no_follow + names:
                if name.module_name.split(".")[0] != function.module_name.split(".")[0]:
                    continue
                if name.module_path is None:
                    raise ValueError(f"Module path not found, for {name}")

                line_numbers = set()
                if name.type == "function":
                    line_numbers = get_function_body_lines(name)
                elif name.type == "param":
                    line_numbers.add(line_number)
                elif (
                    name.type == "statement"
                    or name.type == "class"
                    or name.type == "instance"
                ):
                    line_numbers.update(
                        range(
                            name.get_definition_start_position()[0],
                            name.get_definition_end_position()[0] + 1,
                        )
                    )
                elif name.type == "module":
                    # script_path = name.module_path
                    # context[script_path].update(get_function_body_lines(name))
                    line_numbers.update(get_function_body_lines(name))
                else:
                    print(line_number, column_number)
                    print(name.module_path)
                    raise NotImplementedError(f"Name type {name.type}", name)

                if name.module_path == function.module_path:
                    function_body_lines.update(line_numbers)
                # if not part of current file
                else:
                    context[name.module_path].update(line_numbers)

    context[function.module_path].update(function_body_lines)
    return context


def get_changes_lines_units(
    repo_name: str, file_name: str, fix_changes_line_numbers: list[int]
) -> tuple[str, dict[Path, str]]:
    project = jedi.Project(f"repos/{repo_name}")
    script = jedi.Script(path=f"repos/{repo_name}/{file_name}", project=project)

    functions_body_lines: set[int] = set()
    context_lines: dict[Path, set[int]] = defaultdict(set)
    for fix_line in fix_changes_line_numbers:
        if fix_line in functions_body_lines:
            continue
        line_context = script.get_context(fix_line)
        if line_context.type == "class":
            functions_body_lines.update(get_function_body_lines(line_context))
            for context_file, context_line_numbers in get_function_context(
                script, line_context
            ).items():
                context_lines[context_file].update(context_line_numbers)
        elif line_context.type == "function":
            functions_body_lines.update(get_function_body_lines(line_context))
            for context_file, context_line_numbers in get_function_context(
                script, line_context
            ).items():
                context_lines[context_file].update(context_line_numbers)
        else:
            functions_body_lines.add(fix_line)

    code_unit_data = "\n".join(
        read_lines(f"repos/{repo_name}/{file_name}", functions_body_lines)
    )
    context_data = {
        Path(file): "\n".join(read_lines(str(file), lines))
        for file, lines in context_lines.items()
    }

    return code_unit_data, context_data


# code_unit_changes: list[dict[str, Any]] = []
# code_context_changes: list[dict[str, Any]] = []
repos: dict[str, Repo] = {}

grouped_vulnerabilities = python_vulnerability_fixes.group_by(
    "repo", "vulnerability_id", "commit", "commit_source"
).agg(pl.col("patch"), pl.col("file"), pl.col("language")).sort(by="vulnerability_id")
errors: list[dict[str, Any]] = []
checked_commits = set([change["commit"] for change in code_unit_changes])

for commit_data_row in grouped_vulnerabilities.iter_rows(named=True):
    fix_commit = commit_data_row["commit"]
    if fix_commit in checked_commits:
        continue
    repo_name = commit_data_row["repo"]

    try:
        if not (local_repo := repos.get(repo_name)):
            if os.path.exists(f"repos/{repo_name}"):
                local_repo = Repo(f"repos/{repo_name}")
            elif commit_data_row["commit_source"] == "github":
                repo = github_client.get_repo(repo_name)
                local_repo = Repo.clone_from(repo.clone_url, f"repos/{repo_name}")
            elif commit_data_row["commit_source"] == "gitlab":
                repo = gl.projects.get(repo_name)
                local_repo = Repo.clone_from(repo.ssh_url_to_repo, f"repos/{repo_name}")
            repos[repo_name] = local_repo

        fix_commit, previous_commit = list(
            local_repo.iter_commits(fix_commit, max_count=2)
        )[:2]
        new_old_file = {}
        for change in fix_commit.diff(previous_commit):
            new_old_file[change.a_path] = change.b_path

        for patch, file, language in zip(
            commit_data_row["patch"],
            commit_data_row["file"],
            commit_data_row["language"],
        ):
            diffs = whatthepatch.parse_patch(patch)

            fix_changes_line_numbers: list[int] = []
            previous_changes_line_numbers: list[int] = []
            for diff in diffs:
                for change in diff.changes:
                    if change.new is None:
                        previous_changes_line_numbers.append(change.old)
                    elif change.old is None:
                        fix_changes_line_numbers.append(change.new)
                    # Ignore unchanged lines

            old_file = new_old_file[file]
            local_repo.git.checkout(fix_commit)

            if language == "Python":
                code_unit_after_fix, code_context_after_fix = get_changes_lines_units(
                    repo_name, file, fix_changes_line_numbers
                )

                local_repo.git.checkout(previous_commit)
                code_unit_before_fix, code_context_before_fix = get_changes_lines_units(
                    repo_name, old_file, previous_changes_line_numbers
                )

                code_unit_changes.append(
                    {
                        "commit": fix_commit,
                        "repo": repo_name,
                        "old_file": old_file,
                        "new_file": file,
                        "patch": patch,
                        "code_unit_before_fix": clear_file_content(
                            code_unit_before_fix
                        ),
                        "code_unit_after_fix": clear_file_content(code_unit_after_fix),
                    }
                )
                code_context_changes.append(
                    {
                        "commit": fix_commit,
                        "repo": repo_name,
                        "old_file": old_file,
                        "new_file": file,
                        "patch": patch,
                        "code_context_before_fix": {
                            str(file): clear_file_content(content)
                            for file, content in code_context_before_fix.items()
                        },
                        "code_context_after_fix": {
                            str(file): clear_file_content(content)
                            for file, content in code_context_after_fix.items()
                        },
                    }
                )
            else:
                code_unit_after_fix = clear_file_content(
                    "\n".join(
                        read_lines(
                            f"repos/{repo_name}/{file}", set(fix_changes_line_numbers)
                        )
                    )
                )

                local_repo.git.checkout(previous_commit)
                code_unit_before_fix = clear_file_content(
                    "\n".join(
                        read_lines(
                            f"repos/{repo_name}/{old_file}",
                            set(previous_changes_line_numbers),
                        )
                    )
                )

                code_unit_changes.append(
                    {
                        "commit": fix_commit,
                        "repo": repo_name,
                        "old_file": old_file,
                        "new_file": file,
                        "patch": patch,
                        "code_unit_before_fix": code_unit_before_fix,
                        "code_unit_after_fix": code_unit_after_fix,
                    }
                )
                code_context_changes.append(
                    {
                        "commit": fix_commit,
                        "repo": repo_name,
                        "old_file": old_file,
                        "new_file": file,
                        "patch": patch,
                        "code_context_before_fix": {old_file: code_unit_before_fix},
                        "code_context_after_fix": {file: code_unit_after_fix},
                    }
                )

    except Exception as e:
        print(traceback.print_exception(e))
        errors.append(
            {
                "commit": fix_commit,
                "repo": repo_name,
                "file": file,
                "error": str(e),
            }
        )

Traceback (most recent call last):
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 212, in <module>
    code_unit_after_fix, code_context_after_fix = get_changes_lines_units(
                                                  ~~~~~~~~~~~~~~~~~~~~~~~^
        repo_name, file, fix_changes_line_numbers
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 146, in get_changes_lines_units
    read_lines(f"repos/{repo_name}/{file_name}", functions_body_lines)
    ~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 11, in read_lines
    with open(file) as f:
         ~~~~^^^^^^
  File "/Users/somen/Zavodi/unik/open_source_code_vulnerabilities_dataset/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 324, in _modified

None


Traceback (most recent call last):
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 212, in <module>
    code_unit_after_fix, code_context_after_fix = get_changes_lines_units(
                                                  ~~~~~~~~~~~~~~~~~~~~~~~^
        repo_name, file, fix_changes_line_numbers
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 149, in get_changes_lines_units
    Path(file): "\n".join(read_lines(str(file), lines))
                          ~~~~~~~~~~^^^^^^^^^^^^^^^^^^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 11, in read_lines
    with open(file) as f:
         ~~~~^^^^^^
  File "/Users/somen/Zavodi/unik/open_source_code_vulnerabilities_dataset/.venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py", line 324, in _modified_open
    return io_open(file, 

None


Traceback (most recent call last):
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 212, in <module>
    code_unit_after_fix, code_context_after_fix = get_changes_lines_units(
                                                  ~~~~~~~~~~~~~~~~~~~~~~~^
        repo_name, file, fix_changes_line_numbers
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 138, in get_changes_lines_units
    for context_file, context_line_numbers in get_function_context(
                                              ~~~~~~~~~~~~~~~~~~~~^
        script, line_context
        ^^^^^^^^^^^^^^^^^^^^
    ).items():
    ^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 95, in get_function_context
    name.get_definition_start_position()[0],
    ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subs

...

None


Traceback (most recent call last):
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 212, in <module>
    code_unit_after_fix, code_context_after_fix = get_changes_lines_units(
                                                  ~~~~~~~~~~~~~~~~~~~~~~~^
        repo_name, file, fix_changes_line_numbers
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 138, in get_changes_lines_units
    for context_file, context_line_numbers in get_function_context(
                                              ~~~~~~~~~~~~~~~~~~~~^
        script, line_context
        ^^^^^^^^^^^^^^^^^^^^
    ).items():
    ^
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 69, in get_function_context
    names: list[jedi.api.classes.Name] = script.goto(
                                         ~~~~~~~~~~~^
        line_number, 

None


Traceback (most recent call last):
  File "/Users/somen/Zavodi/unik/open_source_code_vulnerabilities_dataset/.venv/lib/python3.13/site-packages/jedi/cache.py", line 110, in wrapper
KeyError: ((), frozenset())

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/somen/Zavodi/unik/open_source_code_vulnerabilities_dataset/.venv/lib/python3.13/site-packages/jedi/cache.py", line 110, in wrapper
KeyError: ((), frozenset())

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/var/folders/cy/xkclg4xd0gjglf9yxn33fv6r0000gn/T/ipykernel_45489/1899541185.py", line 212, in <module>
    code_unit_after_fix, code_context_after_fix = get_changes_lines_units(
                                                  ~~~~~~~~~~~~~~~~~~~~~~~^
        repo_name, file, fix_changes_line_numbers
        ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
    )
    ^
  File "/var/folders/cy/xkclg4xd0g

None


In [1]:
checked_commits = set([change["commit"] for change in code_unit_changes])

NameError: name 'code_unit_changes' is not defined

In [30]:
print(repo_name)
print(fix_commit)

kovidgoyal/calibre
3a89718664cb8c


In [26]:
diffs = whatthepatch.parse_patch(patch)
list(diffs)[0].changes

[Change(old=177, new=177, line='        pass', hunk=1),
 Change(old=178, new=178, line='', hunk=1),
 Change(old=179, new=179, line='    def skip(self, type):', hunk=1),
 Change(old=180, new=None, line='        if type == TType.STOP:', hunk=1),
 Change(old=181, new=None, line='            return', hunk=1),
 Change(old=182, new=None, line='        elif type == TType.BOOL:', hunk=1),
 Change(old=None, new=180, line='        if type == TType.BOOL:', hunk=1),
 Change(old=183, new=181, line='            self.readBool()', hunk=1),
 Change(old=184, new=182, line='        elif type == TType.BYTE:', hunk=1),
 Change(old=185, new=183, line='            self.readByte()', hunk=1),
 Change(old=220, new=218, line='            for _ in range(size):', hunk=2),
 Change(old=221, new=219, line='                self.skip(etype)', hunk=2),
 Change(old=222, new=220, line='            self.readListEnd()', hunk=2),
 Change(old=None, new=221, line='        else:', hunk=2),
 Change(old=None, new=222, line='     

In [14]:
project = jedi.Project("repos/parisneo/lollms")
script = jedi.Script(
    path="repos/parisneo/lollms/lollms/server/endpoints/lollms_file_system.py", project=project)
name = script.goto(328, 23, follow_imports=True)[0]
name.type

'class'

In [304]:
script.get_context(27).parent().parent()

In [260]:
[name.module_name for name in script.goto(27, 32)]

['tornado.web']

In [214]:
[name for name in script.get_references(53, len("            callback = sanitize_")) if name.is_definition()]

[<Name full_name='code.autocomplete.server.sanitize_callback', description='def sanitize_callback'>]

In [37]:
commit_data_only_top_langs.unique(["cve_id", "commit"]).filter(pl.col("commit") == "63cde2daadc705edf086f2213b48c8c547f98358").head(50)

cve_id,repo,commit,language,reference,year,cwe_id,detection_date,score
str,str,str,str,str,i64,str,str,f64
"""2021-21695""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-59""","""2022-05-24T19:19:43.000000""",9.0
"""2021-21690""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-22""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21694""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-862""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21688""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-862""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21692""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-22""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21686""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-22""","""2022-05-24T19:19:45.000000""",9.0
"""2021-21693""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-285""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21691""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-59""","""2022-05-24T19:19:44.000000""",9.0
"""2021-21689""","""jenkinsci/jenkins""","""63cde2daadc705edf086f2213b48c8…","""Java""","""https://github.com/jenkinsci/j…",2021,"""CWE-862""","""2022-05-24T19:19:44.000000""",9.0


In [258]:
another_script = jedi.Script(path="repos/rtxteam/rtx/code/kg2c/create_indexes_constraints_canonicalized.py", project=project)
another_script.goto(102,36)

[]